In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2D_MaxPool2D = nn.Sequential(
            nn.Conv2d(3, 30, (5,5), stride=2),
            nn.MaxPool2d((1,1)),
            nn.Conv2d(30, 60, (2,2), stride=3),
            nn.MaxPool2d((1,1)),
            nn.ReLU(),
            nn.Conv2d(60, 120, (5,5), stride=4),
            nn.MaxPool2d((1,1)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(9720, 1000),
        )

    def forward(self, x):
        x = self.conv2D_MaxPool2D(x)
        return x
    

transform = transforms.Compose([
    transforms.Resize((244, 244)), # Redimensionar a 200x200p
    transforms.ToTensor(), # Convertir imatges a tensors de PyTorch
])

data_path = '/home/itibcn/Desktop/Torch/datasets/pokemondataset/train'
data_path2 = '/home/itibcn/Desktop/Torch/datasets/pokemondataset/test'

train = datasets.ImageFolder(root=data_path, transform=transform)
test = datasets.ImageFolder(root=data_path2, transform = transform)

dataloader_train = DataLoader(train, batch_size=10, shuffle=True)
dataloader_test = DataLoader(test, batch_size=10, shuffle=True)

output_dimension = len(datasets.ImageFolder(root=data_path).classes)
bs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10

#Inicialitzar el model, funcio de perdua i optimitzador
model = NeuralNetwork().to(device)

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Mostrar las clases
print(f"Clases del dataset: {train.classes}")
print(len(train.classes))

def train_loop(dataloader, model, loss_fn, optimizer, bs):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * bs + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:

            X = X.to(device)
            y = y.to(device)    
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


for epoch in range(epochs):
    
    print("Epoch: " + str(epoch))
    train_loop(dataloader_train, model, loss, optimizer, bs)
    test_loop(dataloader_test, model, loss)

    #epoch_loss = running_loss / len(train)
    #epoch_accuracy = 100 * correct / total
    #print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
    
print('Entrenament finalitzat.')


#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

model_path = '/home/itibcn/Desktop/Torch/modelPokimon.pth'
torch.save(model.state_dict(), model_path)




Clases del dataset: ['abomasnow', 'abra', 'absol', 'accelgor', 'aegislash-shield', 'aerodactyl', 'aggron', 'aipom', 'alakazam', 'alcremie', 'alomomola', 'altaria', 'amaura', 'ambipom', 'amoonguss', 'ampharos', 'annihilape', 'anorith', 'appletun', 'applin', 'araquanid', 'arbok', 'arboliva', 'arcanine', 'arceus', 'archen', 'archeops', 'arctibax', 'arctovish', 'arctozolt', 'ariados', 'armaldo', 'armarouge', 'aromatisse', 'aron', 'arrokuda', 'articuno', 'audino', 'aurorus', 'avalugg', 'axew', 'azelf', 'azumarill', 'azurill', 'bagon', 'baltoy', 'banette', 'barbaracle', 'barboach', 'barraskewda', 'basculegion-male', 'basculin-red-striped', 'bastiodon', 'baxcalibur', 'bayleef', 'beartic', 'beautifly', 'beedrill', 'beheeyem', 'beldum', 'bellibolt', 'bellossom', 'bellsprout', 'bergmite', 'bewear', 'bibarel', 'bidoof', 'binacle', 'bisharp', 'blacephalon', 'blastoise', 'blaziken', 'blipbug', 'blissey', 'blitzle', 'boldore', 'boltund', 'bombirdier', 'bonsly', 'bouffalant', 'bounsweet', 'braixen', 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x9720 and 9721x1000)

In [20]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Define nuevamente la clase NeuralNetwork
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv2D_MaxPool2D = nn.Sequential(
            nn.Conv2d(3, 30, (5, 5), stride=2),
            nn.MaxPool2d((1, 1)),
            nn.Conv2d(30, 60, (2, 2), stride=3),
            nn.MaxPool2d((1, 1)),
            nn.ReLU(),
            nn.Conv2d(60, 120, (5, 5), stride=4),
            nn.MaxPool2d((1, 1)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(9720, 1000),
        )

    def forward(self, x):
        x = self.conv2D_MaxPool2D(x)
        return x

# Función para cargar y predecir
def predict_pokemon(image_path, model, classes, device):
    """
    Predice el Pokémon en una imagen dada utilizando el modelo entrenado.

    :param image_path: Ruta a la imagen del Pokémon
    :param model: Modelo preentrenado para clasificar Pokémon
    :param classes: Lista de nombres de clases (Pokémon)
    :param device: CPU o GPU donde se ejecutará la predicción
    :return: Nombre del Pokémon predicho
    """
    transform = transforms.Compose([
        transforms.Resize((244, 244)),  # Tamaño de entrada según el entrenamiento
        transforms.ToTensor(),         
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalización
    ])

    # Procesa la imagen
    try:
        image = Image.open(image_path).convert('RGB')  # Convertir a RGB
        image = transform(image).unsqueeze(0)         # Añadir la dimensión batch
    except Exception as e:
        print(f"Error al cargar la imagen: {e}")
        return None

    image = image.to(device)

    # Realiza la predicción
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_class = classes[predicted.item()]

    return predicted_class

# Ruta al modelo guardado
model_path = "modelPokimon.pth" #Pesos del model entrenat guardats en aquesta variable

# Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Clases definidas en el entrenamiento
classes = ['abomasnow', 'abra', 'absol', 'accelgor', 'aegislash-shield', 'aerodactyl', 'aggron', 'aipom', 'alakazam', 'alcremie', 'alomomola', 'altaria', 'amaura', 'ambipom', 'amoonguss', 'ampharos', 'annihilape', 'anorith', 'appletun', 'applin', 'araquanid', 'arbok', 'arboliva', 'arcanine', 'arceus', 'archen', 'archeops', 'arctibax', 'arctovish', 'arctozolt', 'ariados', 'armaldo', 'armarouge', 'aromatisse', 'aron', 'arrokuda', 'articuno', 'audino', 'aurorus', 'avalugg', 'axew', 'azelf', 'azumarill', 'azurill', 'bagon', 'baltoy', 'banette', 'barbaracle', 'barboach', 'barraskewda', 'basculegion-male', 'basculin-red-striped', 'bastiodon', 'baxcalibur', 'bayleef', 'beartic', 'beautifly', 'beedrill', 'beheeyem', 'beldum', 'bellibolt', 'bellossom', 'bellsprout', 'bergmite', 'bewear', 'bibarel', 'bidoof', 'binacle', 'bisharp', 'blacephalon', 'blastoise', 'blaziken', 'blipbug', 'blissey', 'blitzle', 'boldore', 'boltund', 'bombirdier', 'bonsly', 'bouffalant', 'bounsweet', 'braixen', 'brambleghast', 'bramblin', 'braviary', 'breloom', 'brionne', 'bronzong', 'bronzor', 'brute-bonnet', 'bruxish', 'budew', 'buizel', 'bulbasaur', 'buneary', 'bunnelby', 'burmy', 'butterfree', 'buzzwole', 'cacnea', 'cacturne', 'calyrex', 'camerupt', 'capsakid', 'carbink', 'carkol', 'carnivine', 'carracosta', 'carvanha', 'cascoon', 'castform', 'caterpie', 'celebi', 'celesteela', 'centiskorch', 'ceruledge', 'cetitan', 'cetoddle', 'chandelure', 'chansey', 'charcadet', 'charizard', 'charjabug', 'charmander', 'charmeleon', 'chatot', 'cherrim', 'cherubi', 'chesnaught', 'chespin', 'chewtle', 'chikorita', 'chimchar', 'chimecho', 'chinchou', 'chingling', 'cinccino', 'cinderace', 'clamperl', 'clauncher', 'clawitzer', 'claydol', 'clefable', 'clefairy', 'cleffa', 'clobbopus', 'clodsire', 'cloyster', 'coalossal', 'cobalion', 'cofagrigus', 'combee', 'combusken', 'comfey', 'conkeldurr', 'copperajah', 'corphish', 'corsola', 'corviknight', 'corvisquire', 'cosmoem', 'cosmog', 'cottonee', 'crabominable', 'crabrawler', 'cradily', 'cramorant', 'cranidos', 'crawdaunt', 'cresselia', 'croagunk', 'crobat', 'crocalor', 'croconaw', 'crustle', 'cryogonal', 'cubchoo', 'cubone', 'cufant', 'cursola', 'cutiefly', 'cyclizar', 'cyndaquil', 'dachsbun', 'darkrai', 'darmanitan-standard', 'dartrix', 'darumaka', 'decidueye', 'dedenne', 'deerling', 'deino', 'delcatty', 'delibird', 'delphox', 'deoxys-normal', 'dewgong', 'dewott', 'dewpider', 'dhelmise', 'dialga', 'diancie', 'diggersby', 'diglett', 'ditto', 'dodrio', 'doduo', 'dolliv', 'dondozo', 'donphan', 'dottler', 'doublade', 'dracovish', 'dracozolt', 'dragalge', 'dragapult', 'dragonair', 'dragonite', 'drakloak', 'drampa', 'drapion', 'dratini', 'drednaw', 'dreepy', 'drifblim', 'drifloon', 'drilbur', 'drizzile', 'drowzee', 'druddigon', 'dubwool', 'ducklett', 'dudunsparce-two-segment', 'dugtrio', 'dunsparce', 'duosion', 'duraludon', 'durant', 'dusclops', 'dusknoir', 'duskull', 'dustox', 'dwebble', 'eelektrik', 'eelektross', 'eevee', 'eiscue-ice', 'ekans', 'eldegoss', 'electabuzz', 'electivire', 'electrike', 'electrode', 'elekid', 'elgyem', 'emboar', 'emolga', 'empoleon', 'enamorus-incarnate', 'entei', 'escavalier', 'espathra', 'espeon', 'espurr', 'eternatus', 'excadrill', 'exeggcute', 'exeggutor', 'exploud', 'falinks', 'farfetchd', 'farigiraf', 'fearow', 'feebas', 'fennekin', 'feraligatr', 'ferroseed', 'ferrothorn', 'fidough', 'finizen', 'finneon', 'flaaffy', 'flabebe', 'flamigo', 'flapple', 'flareon', 'fletchinder', 'fletchling', 'flittle', 'floatzel', 'floette', 'floragato', 'florges', 'flutter-mane', 'flygon', 'fomantis', 'foongus', 'forretress', 'fraxure', 'frigibax', 'frillish', 'froakie', 'frogadier', 'froslass', 'frosmoth', 'fuecoco', 'furfrou', 'furret', 'gabite', 'gallade', 'galvantula', 'garbodor', 'garchomp', 'gardevoir', 'garganacl', 'gastly', 'gastrodon', 'genesect', 'gengar', 'geodude', 'gholdengo', 'gible', 'gigalith', 'gimmighoul', 'girafarig', 'giratina-altered', 'glaceon', 'glalie', 'glameow', 'glastrier', 'gligar', 'glimmet', 'glimmora', 'gliscor', 'gloom', 'gogoat', 'golbat', 'goldeen', 'golduck', 'golem', 'golett', 'golisopod', 'golurk', 'goodra', 'goomy', 'gorebyss', 'gossifleur', 'gothita', 'gothitelle', 'gothorita', 'gourgeist-average', 'grafaiai', 'granbull', 'grapploct', 'graveler', 'great-tusk', 'greavard', 'greedent', 'greninja', 'grimer', 'grimmsnarl', 'grookey', 'grotle', 'groudon', 'grovyle', 'growlithe', 'grubbin', 'grumpig', 'gulpin', 'gumshoos', 'gurdurr', 'guzzlord', 'gyarados', 'hakamo-o', 'happiny', 'hariyama', 'hatenna', 'hatterene', 'hattrem', 'haunter', 'hawlucha', 'haxorus', 'heatmor', 'heatran', 'heliolisk', 'helioptile', 'heracross', 'herdier', 'hippopotas', 'hippowdon', 'hitmonchan', 'hitmonlee', 'hitmontop', 'ho-oh', 'honchkrow', 'honedge', 'hoopa', 'hoothoot', 'hoppip', 'horsea', 'houndoom', 'houndour', 'houndstone', 'huntail', 'hydreigon', 'hypno', 'igglybuff', 'illumise', 'impidimp', 'incineroar', 'indeedee-male', 'infernape', 'inkay', 'inteleon', 'iron-bundle', 'iron-hands', 'iron-jugulis', 'iron-moth', 'iron-thorns', 'iron-treads', 'ivysaur', 'jangmo-o', 'jellicent', 'jigglypuff', 'jirachi', 'jolteon', 'joltik', 'jumpluff', 'jynx', 'kabuto', 'kabutops', 'kadabra', 'kakuna', 'kangaskhan', 'karrablast', 'kartana', 'kecleon', 'keldeo-ordinary', 'kilowattrel', 'kingambit', 'kingdra', 'kingler', 'kirlia', 'klang', 'klawf', 'kleavor', 'klefki', 'klink', 'klinklang', 'koffing', 'komala', 'kommo-o', 'krabby', 'kricketot', 'kricketune', 'krokorok', 'krookodile', 'kubfu', 'kyogre', 'kyurem', 'lairon', 'lampent', 'landorus-incarnate', 'lanturn', 'lapras', 'larvesta', 'larvitar', 'latias', 'latios', 'leafeon', 'leavanny', 'lechonk', 'ledian', 'ledyba', 'lickilicky', 'lickitung', 'liepard', 'lileep', 'lilligant', 'lillipup', 'linoone', 'litleo', 'litten', 'litwick', 'lokix', 'lombre', 'lopunny', 'lotad', 'loudred', 'lucario', 'ludicolo', 'lugia', 'lumineon', 'lunala', 'lunatone', 'lurantis', 'luvdisc', 'luxio', 'luxray', 'lycanroc-midday', 'mabosstiff', 'machamp', 'machoke', 'machop', 'magby', 'magcargo', 'magearna', 'magikarp', 'magmar', 'magmortar', 'magnemite', 'magneton', 'magnezone', 'makuhita', 'malamar', 'mamoswine', 'manaphy', 'mandibuzz', 'manectric', 'mankey', 'mantine', 'mantyke', 'maractus', 'mareanie', 'mareep', 'marill', 'marowak', 'marshadow', 'marshtomp', 'maschiff', 'masquerain', 'maushold-family-of-four', 'mawile', 'medicham', 'meditite', 'meganium', 'melmetal', 'meloetta-aria', 'meltan', 'meowscarada', 'meowstic-male', 'meowth', 'mesprit', 'metagross', 'metang', 'metapod', 'mew', 'mewtwo', 'mienfoo', 'mienshao', 'mightyena', 'milcery', 'milotic', 'miltank', 'mime-jr', 'mimikyu-disguised', 'minccino', 'minior-red-meteor', 'minun', 'misdreavus', 'mismagius', 'moltres', 'monferno', 'morelull', 'morgrem', 'morpeko-full-belly', 'mothim', 'mr-mime', 'mr-rime', 'mudbray', 'mudkip', 'mudsdale', 'muk', 'munchlax', 'munna', 'murkrow', 'musharna', 'nacli', 'naclstack', 'naganadel', 'natu', 'necrozma', 'nickit', 'nidoking', 'nidoqueen', 'nidoran-f', 'nidoran-m', 'nidorina', 'nidorino', 'nihilego', 'nincada', 'ninetales', 'ninjask', 'noctowl', 'noibat', 'noivern', 'nosepass', 'numel', 'nuzleaf', 'nymble', 'obstagoon', 'octillery', 'oddish', 'oinkologne-male', 'omanyte', 'omastar', 'onix', 'oranguru', 'orbeetle', 'oricorio-baile', 'orthworm', 'oshawott', 'overqwil', 'pachirisu', 'palafin-zero', 'palkia', 'palossand', 'palpitoad', 'pancham', 'pangoro', 'panpour', 'pansage', 'pansear', 'paras', 'parasect', 'passimian', 'patrat', 'pawmi', 'pawmo', 'pawmot', 'pawniard', 'pelipper', 'perrserker', 'persian', 'petilil', 'phanpy', 'phantump', 'pheromosa', 'phione', 'pichu', 'pidgeot', 'pidgeotto', 'pidgey', 'pidove', 'pignite', 'pikachu', 'pikipek', 'piloswine', 'pincurchin', 'pineco', 'pinsir', 'piplup', 'plusle', 'poipole', 'politoed', 'poliwag', 'poliwhirl', 'poliwrath', 'polteageist', 'ponyta', 'poochyena', 'popplio', 'porygon', 'porygon-z', 'porygon2', 'primarina', 'primeape', 'prinplup', 'probopass', 'psyduck', 'pumpkaboo-average', 'pupitar', 'purrloin', 'purugly', 'pyroar', 'pyukumuku', 'quagsire', 'quaquaval', 'quaxly', 'quaxwell', 'quilava', 'quilladin', 'qwilfish', 'raboot', 'rabsca', 'raichu', 'raikou', 'ralts', 'rampardos', 'rapidash', 'raticate', 'rattata', 'rayquaza', 'regice', 'regidrago', 'regieleki', 'regigigas', 'regirock', 'registeel', 'relicanth', 'rellor', 'remoraid', 'reshiram', 'reuniclus', 'revavroom', 'rhydon', 'rhyhorn', 'rhyperior', 'ribombee', 'rillaboom', 'riolu', 'rockruff', 'roggenrola', 'rolycoly', 'rookidee', 'roselia', 'roserade', 'rotom', 'rowlet', 'rufflet', 'runerigus', 'sableye', 'salamence', 'salandit', 'salazzle', 'samurott', 'sandaconda', 'sandile', 'sandshrew', 'sandslash', 'sandy-shocks', 'sandygast', 'sawk', 'sawsbuck', 'scatterbug', 'sceptile', 'scizor', 'scolipede', 'scorbunny', 'scovillain', 'scrafty', 'scraggy', 'scream-tail', 'scyther', 'seadra', 'seaking', 'sealeo', 'seedot', 'seel', 'seismitoad', 'sentret', 'serperior', 'servine', 'seviper', 'sewaddle', 'sharpedo', 'shaymin-land', 'shedinja', 'shelgon', 'shellder', 'shellos', 'shelmet', 'shieldon', 'shiftry', 'shiinotic', 'shinx', 'shroodle', 'shroomish', 'shuckle', 'shuppet', 'sigilyph', 'silcoon', 'silicobra', 'silvally', 'simipour', 'simisage', 'simisear', 'sinistea', 'sirfetchd', 'sizzlipede', 'skarmory', 'skeledirge', 'skiddo', 'skiploom', 'skitty', 'skorupi', 'skrelp', 'skuntank', 'skwovet', 'slaking', 'slakoth', 'sliggoo', 'slither-wing', 'slowbro', 'slowking', 'slowpoke', 'slugma', 'slurpuff', 'smeargle', 'smoliv', 'smoochum', 'sneasel', 'sneasler', 'snivy', 'snom', 'snorlax', 'snorunt', 'snover', 'snubbull', 'sobble', 'solgaleo', 'solosis', 'solrock', 'spearow', 'spectrier', 'spewpa', 'spheal', 'spidops', 'spinarak', 'spinda', 'spiritomb', 'spoink', 'sprigatito', 'spritzee', 'squawkabilly-green-plumage', 'squirtle', 'stakataka', 'stantler', 'staraptor', 'staravia', 'starly', 'starmie', 'staryu', 'steelix', 'steenee', 'stonjourner', 'stoutland', 'stufful', 'stunfisk', 'stunky', 'sudowoodo', 'suicune', 'sunflora', 'sunkern', 'surskit', 'swablu', 'swadloon', 'swalot', 'swampert', 'swanna', 'swellow', 'swinub', 'swirlix', 'swoobat', 'sylveon', 'tadbulb', 'taillow', 'talonflame', 'tandemaus', 'tangela', 'tangrowth', 'tapu-bulu', 'tapu-fini', 'tapu-koko', 'tapu-lele', 'tarountula', 'tatsugiri-curly', 'tauros', 'teddiursa', 'tentacool', 'tentacruel', 'tepig', 'terrakion', 'thievul', 'throh', 'thundurus-incarnate', 'thwackey', 'timburr', 'tinkatink', 'tinkaton', 'tinkatuff', 'tirtouga', 'toedscool', 'toedscruel', 'togedemaru', 'togekiss', 'togepi', 'togetic', 'torchic', 'torkoal', 'tornadus-incarnate', 'torracat', 'torterra', 'totodile', 'toucannon', 'toxapex', 'toxel', 'toxicroak', 'toxtricity-amped', 'tranquill', 'trapinch', 'treecko', 'trevenant', 'tropius', 'trubbish', 'trumbeak', 'tsareena', 'turtonator', 'turtwig', 'tympole', 'tynamo', 'type-null', 'typhlosion', 'tyranitar', 'tyrantrum', 'tyrogue', 'tyrunt', 'umbreon', 'unfezant', 'unown', 'ursaluna', 'ursaring', 'urshifu-single-strike', 'uxie', 'vanillish', 'vanillite', 'vanilluxe', 'vaporeon', 'varoom', 'veluza', 'venipede', 'venomoth', 'venonat', 'venusaur', 'vespiquen', 'vibrava', 'victini', 'victreebel', 'vigoroth', 'vikavolt', 'vileplume', 'virizion', 'vivillon', 'volbeat', 'volcanion', 'volcarona', 'voltorb', 'vullaby', 'vulpix', 'wailmer', 'wailord', 'walrein', 'wartortle', 'watchog', 'wattrel', 'weavile', 'weedle', 'weepinbell', 'weezing', 'whimsicott', 'whirlipede', 'whiscash', 'whismur', 'wigglytuff', 'wiglett', 'wimpod', 'wingull', 'wishiwashi-solo', 'wobbuffet', 'woobat', 'wooloo', 'wooper', 'wormadam-plant', 'wugtrio', 'wurmple', 'wynaut', 'wyrdeer', 'xatu', 'xerneas', 'xurkitree', 'yamask', 'yamper', 'yanma', 'yanmega', 'yungoos', 'yveltal', 'zacian', 'zamazenta', 'zangoose', 'zapdos', 'zarude', 'zebstrika', 'zekrom', 'zeraora', 'zigzagoon', 'zoroark', 'zorua', 'zubat', 'zweilous', 'zygarde-50']


# Carga del modelo
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_path))
print("Modelo cargado exitosamente.")

# Ruta de la imagen
image_path = "/home/itibcn/Desktop/Torch/ModelsEntrenats/AdivinaPokemon/alakazam.png"

# Predicción
predicted_pokemon = predict_pokemon(image_path, model, classes, device)
print(f"El Pokémon predicho es: {predicted_pokemon}")


Modelo cargado exitosamente.
El Pokémon predicho es: jolteon


/tmp/ipykernel_6711/1923221178.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
